In [67]:
import os
import tensorflow as tf
import numpy as np
import time
from tf_utils import input_fn_from_dataset,input_fn_frame_from_dataset,save_tf_record,prob_positive_class_from_prediction
from get_data import get_videos_from_folder,get_target_from_csv
from utils import save_solution
from keras.layers import Dense, Flatten, Dropout, ZeroPadding3D
from keras.layers.recurrent import LSTM
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import TimeDistributed
from keras.layers.convolutional import (Conv2D, MaxPooling3D, Conv3D,
                                        MaxPooling2D)
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger
from sklearn.metrics import roc_auc_score
from keras import backend as K

In [68]:
# AUC for a binary classifier
def auc(y_true, y_pred):
    ptas = tf.stack([binary_PTA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.stack([binary_PFA(y_true,y_pred,k) for k in np.linspace(0, 1, 1000)],axis=0)
    pfas = tf.concat([tf.ones((1,)) ,pfas],axis=0)
    binSizes = -(pfas[1:]-pfas[:-1])
    s = ptas*binSizes
    return K.sum(s, axis=0)
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# PFA, prob false alert for binary classifier
def binary_PFA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # N = total number of negative labels
    N = K.sum(1 - y_true)
    # FP = total number of false alerts, alerts from the negative class labels
    FP = K.sum(y_pred - y_pred * y_true)
    return FP/N
#-----------------------------------------------------------------------------------------------------------------------------------------------------
# P_TA prob true alerts for binary classifier
def binary_PTA(y_true, y_pred, threshold=K.variable(value=0.5)):
    y_pred = K.cast(y_pred >= threshold, 'float32')
    # P = total number of positive labels
    P = K.sum(y_true)
    # TP = total number of correct alerts, alerts from the positive class labels
    TP = K.sum(y_pred * y_true)
    return TP/P

In [10]:
train_folder = os.path.join("../train/")
test_folder = os.path.join("../test/")

train_target = os.path.join('../train_target.csv')
my_solution_file = os.path.join('../solution.csv')

x_train = get_videos_from_folder(train_folder)
y_train = get_target_from_csv(train_target)

In [86]:
# maximum number of frames used as input
pframe=200 #max([len(x) for x in x_train])
X = np.zeros((x_train.shape[0], pframe, 100, 100,1))
for i in range(x_train.shape[0]):
    if x_train[i].shape[0] >= pframe:
        X[i,: pframe,:x_train[i].shape[1],:x_train[i].shape[2],0] = x_train[i][: pframe,:x_train[i].shape[1],:x_train[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_train[i].shape[0], x_train[i].shape[1], x_train[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X[i,pos:(pos+frames),:height,:width,0] = x_train[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X[i,pos:pframe,:height,:width,0] = x_train[i][:(pframe-pos),:height,:width]

In [76]:
y = np.array([0,1] if y_train[0] else [1,0])
for l in y_train[1:]:
    if l == 1:
        y = np.vstack([y, [0,1]])
    else:
        y = np.vstack([y, [1,0]])
y

array([[0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0,

In [93]:
nb_epoch=25
checkpointer = ModelCheckpoint( filepath=os.path.join('data', 'checkpoints', 'lcrn' + '-' + \
            '.{epoch:03d}-{val_loss:.3f}.hdf5'),
        verbose=1,
        save_best_only=True)

    
tb = TensorBoard(log_dir=os.path.join('data', 'logs', 'lcrn'))
early_stopper = EarlyStopping(patience=15)
timestamp = time.time()
csv_logger = CSVLogger(os.path.join('data', 'logs', 'lcrn' + '-' + 'training-' + str(timestamp) + '.log'))
model = Sequential()
model.add(TimeDistributed(Conv2D(32, (7, 7), strides=(2, 2),activation='relu',padding='same'), input_shape=( pframe, 100, 100,1)))
model.add(TimeDistributed(Conv2D(32, (3,3),
kernel_initializer="he_normal", activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(64, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(128, (3,3), padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(128, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(256, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(Conv2D(512, (3,3),padding='same', activation='relu')))
model.add(TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))
model.add(Dropout(0.5))
model.add(LSTM(1024, return_sequences=False, dropout=0.5))
model.add(Dense(2, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_225 (TimeDi (None, 200, 50, 50, 32)   1600      
_________________________________________________________________
time_distributed_226 (TimeDi (None, 200, 48, 48, 32)   9248      
_________________________________________________________________
time_distributed_227 (TimeDi (None, 200, 24, 24, 32)   0         
_________________________________________________________________
time_distributed_228 (TimeDi (None, 200, 24, 24, 64)   18496     
_________________________________________________________________
time_distributed_229 (TimeDi (None, 200, 24, 24, 64)   36928     
_________________________________________________________________
time_distributed_230 (TimeDi (None, 200, 12, 12, 64)   0         
_________________________________________________________________
time_distributed_231 (TimeDi (None, 200, 12, 12, 128)  73856     
__________

In [ ]:
metrics = [auc, 'accuracy']
optimizer = Adam(lr=1e-5, decay=1e-6)
model.compile(loss='binary_crossentropy', optimizer=optimizer,
                       metrics=metrics)
model.fit(X, y, batch_size=32, validation_split=0.2, verbose=1,callbacks=[tb, early_stopper, csv_logger], epochs=nb_epoch)

Train on 126 samples, validate on 32 samples
Epoch 1/25
126/126 [==============================] - 259s 2s/step - loss: 0.7091 - auc: 0.4840 - acc: 0.4603 - val_loss: 0.7173 - val_auc: 0.4170 - val_acc: 0.4062
Epoch 2/25
126/126 [==============================] - 241s 2s/step - loss: 0.6826 - auc: 0.5642 - acc: 0.5238 - val_loss: 0.7309 - val_auc: 0.4697 - val_acc: 0.4062
Epoch 3/25
126/126 [==============================] - 258s 2s/step - loss: 0.6970 - auc: 0.5435 - acc: 0.5556 - val_loss: 0.7291 - val_auc: 0.4736 - val_acc: 0.4062
Epoch 4/25
126/126 [==============================] - 252s 2s/step - loss: 0.6972 - auc: 0.5309 - acc: 0.5238 - val_loss: 0.7052 - val_auc: 0.4482 - val_acc: 0.4062
Epoch 5/25
126/126 [==============================] - 257s 2s/step - loss: 0.6902 - auc: 0.5608 - acc: 0.5238 - val_loss: 0.6930 - val_auc: 0.4629 - val_acc: 0.3750
Epoch 6/25
126/126 [==============================] - 264s 2s/step - loss: 0.6958 - auc: 0.5159 - acc: 0.4921 - val_loss: 0.6981 -

In [5]:
x_test = get_videos_from_folder(test_folder)

In [90]:
X_test = np.zeros((x_test.shape[0],  pframe, 100, 100,1))
for i in range(x_test.shape[0]):
    if x_test[i].shape[0]>= pframe:
        X_test[i,: pframe,:x_test[i].shape[1],:x_test[i].shape[2],0] = x_test[i][: pframe,:x_test[i].shape[1],:x_test[i].shape[2]]
    else:
        # repeated padding
        frames, height, width = x_test[i].shape[0], x_test[i].shape[1], x_test[i].shape[2]
        pos = 0
        while pos + frames <= pframe:
            X_test[i,pos:(pos+frames),:height,:width,0] = x_test[i][:frames,:height,:width]
            pos += frames
        if pos < pframe:
            X_test[i,pos:pframe,:height,:width,0] = x_test[i][:(pframe-pos),:height,:width]

In [91]:
train_predictions = model.predict(X_test) 


In [92]:
dummy_solution = np.array([p[1] for p in train_predictions])
save_solution(my_solution_file,dummy_solution)